In [1]:
import pandas as pd
import numpy as np
import sys 

sys.path.append("..")

from src.support_cleaning import normalize

In [ ]:
data_2013 = pd.read_csv("../data/datos-2013.csv", sep=";").assign(fichero="data_2013")
data_2013.columns = [normalize(column).replace(" ","_") for column in data_2013.columns]
print("\n\nHead:")
display(data_2013.head(5))
print("\n\nData types:")
data_2013.info()

##### Data types
Data types appear to be correct for most columns except for the following ones that are object datatypes and should instead be:
- codigo_orgao_superior: object/category
- codigo_orgao: object: object
- codigo_unidade_gestora: object
- valor_previsto_atualizado: float  
- valor_lancado: float           
- valor_realizado: float         
- percentual_realizado: float
- data_lancamento: datetime[ns]

The codes although numerical, are actually a unique representation of organizations and bear no magnitude. Thus it is more interesting to change them to object to explore their summary statistics more comfortably. 

The revenue values carry a comma instead of a floating point, making conversion to float not feasible until replaced by a point. 

Data_lancamento will just need to simply be converted to datetime.

This will have to be corrected with in the cleaning notebook with a dictionary of the sort below.

In [4]:
data_types_dict = {
    "codigo_orgao_superior": object,
    "codigo_orgao": object,  
    "codigo_unidade_gestora": object,      
    "valor_previsto_atualizado": float,
    "valor_lancado": float,  
    "valor_realizado": float,      
    "percentual_realizado": float,
    "data_lancamento": "datetime64[ns]"
}

In [ ]:
data_dict = {
    "data_2013": pd.read_csv("../data/datos-2013.csv", sep=";"),
    "data_2014": pd.read_csv("../data/datos-2014.csv", sep=";"),
    "data_2015": pd.read_csv("../data/datos-2015.csv", sep=";"),
    "data_2016": pd.read_csv("../data/datos-2016.csv", sep=";"),
    "data_2017": pd.read_csv("../data/datos-2017.csv", sep=";"),
    "data_2018": pd.read_csv("../data/datos-2018.csv", sep=";"),
    "data_2019": pd.read_csv("../data/datos-2019.csv", sep=";"),
    "data_2020": pd.read_csv("../data/datos-2020.csv", sep=";"),
    "data_2021": pd.read_csv("../data/datos-2021.csv", sep=";")
}

data_df = pd.DataFrame(columns=[normalize(column).replace(" ","_") for column in data_2013.columns])
for df_name, df in data_dict.items():
    df.columns = [normalize(column).replace(" ","_") for column in df.columns]
    print("\n\n\n\n------------------")
    print(f"\n{df_name}")
    print("\n\nInfo summary")
    print(df.info())
    print("\n\nDescriptive statistics summary")
    print("\nNumerical data:")
    display(df.describe().T.assign(missing_values=lambda x: df.shape[0]-x["count"]))
    print("\nObject types:")
    display(df.describe(include=['O']).T.assign(missing_values=lambda x: df.shape[0]-x["count"]))

    print("\n\nDuplicated values:")
    display(df.duplicated().sum())

    data_df = pd.concat([data_df,df.assign(fichero=df_name[-4:])])


The main problem with this dataset is the presence of missing values, specially problematic for the feature 'nome_orgao_superior'. Nonetheless, it seems the value from this column can in many cases be inferred from the codigo_orgao_superior or the nome_orgao column. To check that:

In [ ]:
data_df.loc[:,["nome_orgao_superior","codigo_orgao_superior"]].head()

In [ ]:
nome_orgao_superior_notnull = ~data_df["nome_orgao_superior"].isna()
nome_orgao_notnull = ~data_df["nome_orgao"].isna()
data_df.loc[nome_orgao_superior_notnull & nome_orgao_notnull,["nome_orgao_superior","nome_orgao"]].head()

There are many different organizations and management units, but not so many unique types of economical categories. Details, albeit highly cardinal, seem to be part of fixed categories, at least for every given year.

One other thing to note is that the most frequent values for valor columns are 0, which might mean these are missing values not encoded as such.

In [ ]:
valor_previsto_atualizado_zero = data_df["valor_previsto_atualizado"] == "0,00"
valor_lancado = data_df["valor_lancado"] == "0,00"
valor_realizado = data_df["valor_realizado"] == "0,00"
percentual_realizado = data_df["percentual_realizado"] == "0,00"
data_df.loc[valor_previsto_atualizado_zero | valor_lancado | percentual_realizado | valor_realizado,:]

It seems that NaN values are not encoded as 0 in the end. However, one thing to note is that in many cases the realised value comes from an expected value of 0 and viceversa. That means that there are of course unexpected revenues, but could also mean that there are many unrealised revenues or that sometimes the same account for a revenue divides its registration into two rows. That is something to explore once the data cleaning has been performed.

Speaking of duplicates, not many rows in the dataset are doubled and this will be solved in the cleaning phase.


Appart from those things, a last interesting fact is the increment in available data from the year 2016, maybe pointing to an effort to track public revenue more rigourously.

Printing the concatenated dataframe:

In [ ]:
print("Final concatenated dataframe:")
data_df

Finally, saving the concatenated dataframe to parquet for optimised storage to work with it during the cleaning phase.

In [11]:
data_df.to_parquet("../data/concatenated_data.parquet")

## 2. Exploration of cleaned data

In [ ]:
cleaned_data_df = pd.read_parquet("../data/cleaned_data.parquet")
cleaned_data_df.info()

In [274]:
value_features = ["valor_previsto_atualizado","valor_lancado","valor_realizado","percentual_realizado"]

In [ ]:
cleaned_data_df[value_features].describe().T.assign(nulls= lambda x: cleaned_data_df.shape[0] - x["count"], 
                                                    null_pct=lambda x: x["nulls"]/cleaned_data_df.shape[0]*100)


Per the above descriptive summary it can be concluded that:
- There is an enormous spread in the data for all four features.
- Except for valor_realizado, more than 75% of the values are 0. All four features range by hundreds of millions negative and positive, showing a standard deaviation higher than the mean.
- Nulls percentage is not too high, as it does not reach 5% (an industry standard), however, these missing values could potentially be hiding the most interesting information, that will have to be observed through analysis.

In [ ]:
cleaned_data_df.columns

## 2.1 Missing values

In [277]:
cleaned_data_df["nan_per_row"] = cleaned_data_df.isna().sum(axis=1) / cleaned_data_df.shape[1]

#### Are all zero values also missing values?

In [ ]:
filtro = (cleaned_data_df['valor_previsto_atualizado'] == 0) & (cleaned_data_df['valor_realizado'] == 0) & (cleaned_data_df['valor_lancado'] == 0) & (cleaned_data_df['percentual_realizado'] == 0 )
cleaned_data_df.loc[filtro,["especie_receita","detalhamento"]].value_counts().reset_index()

Nome orgao superior

In [ ]:
(cleaned_data_df.groupby('nome_orgao_superior')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_previsto_atualizado",ascending=False))

In [ ]:
(cleaned_data_df.groupby('nome_orgao_superior')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum()/x.count()*100)
                                                 .sort_values(by="valor_realizado",ascending=False))

Nome orgao

In [ ]:
(cleaned_data_df.groupby('nome_orgao')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False))

nome_unidade_gestora

In [ ]:
(cleaned_data_df.groupby('nome_unidade_gestora')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False))

Categoria economica

In [ ]:
(cleaned_data_df.groupby('categoria_economica')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False))

origem_receita

In [ ]:
(cleaned_data_df.groupby('origem_receita')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False))

especie_receita

In [ ]:
(cleaned_data_df.groupby('especie_receita')[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False))

detalhamento

In [ ]:
(cleaned_data_df.groupby(['origem_receita','especie_receita','detalhamento'])[['valor_previsto_atualizado','valor_lancado',
                                                 'valor_realizado', 'percentual_realizado',]]
                                                 .apply(lambda x: x.isna().sum())
                                                 .sort_values(by="valor_realizado",ascending=False)).head(10)

## 3. EDA

#### Negative values

##### Valor realizado

In [ ]:
cleaned_data_df[cleaned_data_df["valor_realizado"] < 0]

In [ ]:
cleaned_data_df[cleaned_data_df['valor_realizado'] < 0].groupby('nome_orgao_superior')["valor_realizado"].agg(["count","sum"]).reset_index().sort_values(by="sum")

In [ ]:
cleaned_data_df[cleaned_data_df['valor_realizado'] < 0].groupby('ano_exercicio')["valor_realizado"].agg(["count","sum"]).reset_index()

#### valor_previsto_atualizado

In [ ]:
pd.set_option("display.max_rows",50)
pd.set_option("display.max_columns",50)
cleaned_data_df.loc[cleaned_data_df["valor_previsto_atualizado"] < 0,]

##### Valor lancado

In [ ]:
cleaned_data_df[cleaned_data_df['valor_lancado'] < 0].groupby('nome_orgao_superior')["valor_lancado"].sum().reset_index().sort_values(by="valor_lancado")

### EDA

Distribución de Ingresos por Categoría Económica:

Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.

Calcular la diferencia promedio entre ingresos previstos y realizados por cada categoría.

Análisis Temporal:

Evaluar las tendencias a lo largo del tiempo, por ejemplo, cómo cambian los ingresos realizados de un mes a otro o de un año a otro.
Identificación de Discrepancias:

Investigar las categorías con mayor diferencia entre lo previsto y lo realizado, identificando patrones en la subejecución o sobre ejecución.

In [ ]:
cleaned_data_df.columns

### Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.

In [ ]:
cleaned_data_df.groupby(["categoria_economica"])[["valor_realizado"]].sum()

1. **Distribución de Ingresos por Categoría Económica:**

   - Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.


The 3 top contributing revenue categories are: 'operacoes_de_credito_mercado_interno', 'contribuicoes_sociais' and 'impostos'.

In [ ]:
pd.set_option("display.max_rows",65)
contribution_especie = cleaned_data_df.groupby(["especie_receita"])[["valor_realizado"]].sum().assign(valor_realizado_pct=lambda x: np.round(x["valor_realizado"]/cleaned_data_df["valor_realizado"].sum()*100,2)).sort_values("valor_realizado",ascending=False)
contribution_especie[0:10]

   - Calcular la diferencia promedio entre ingresos previstos y realizados por cada categoría.
   


In [ ]:
cleaned_data_df["month"] = cleaned_data_df["data_lancamento"].dt.month
cleaned_data_df["month_name"] = cleaned_data_df["data_lancamento"].dt.month_name() 
cleaned_data_df["register"] = 1

In [ ]:
cleaned_data_df["diff_valor_previsto_realizado"] = cleaned_data_df["valor_realizado"] - cleaned_data_df["valor_previsto_atualizado"]

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'categoria_economica'])['diff_valor_previsto_realizado'].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby('categoria_economica')['diff_valor_previsto_realizado'].mean().reset_index()
mean_yearly_revenue_category.columns = ["categoria_economica","mean_yearly_revenue_diff"]
mean_yearly_revenue_category.sort_values(by="mean_yearly_revenue_diff")

The top revenue contributing categories are also the ones showing the biggest deviation in forecast VS. realised. Although there are categories like 'alienacao_de_bens_imoveis' or 'receita_titulos_do_tesouro_nacional_resgatado' that were expecting more than 20 times more revenue (revenue_diff_valor_realizado_pct), or worse 'demais_recetas_de_capital' that expected hundreds of millions and received zero, the top first 'operacoes_de_credito_mercado_interno' that overforecasts on avegerage by a stunning 36% represents a 92% of the whole overforecasted revenue. 

Similarly, 'contribuicoes_sociais' and 'contribuicoes_sociais', represent respectively a 15% and a 8% of the whole overforecasted revenue. On the other hand, another top revenue contributor, 'resultado_do_banco_central_do_brasil' underforecasts by 50% and brings in a 15% of the overforcasted revenue in form of unexpected revenue.

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'especie_receita'])[['diff_valor_previsto_realizado',"valor_realizado"]].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby('especie_receita')[['diff_valor_previsto_realizado',"valor_realizado"]].mean().reset_index()
mean_yearly_revenue_category.columns = ["especie_receita","mean_yearly_revenue_diff","valor_realizado"]

(mean_yearly_revenue_category.assign(revenue_diff_pct= lambda x: np.round(x["mean_yearly_revenue_diff"]/x["mean_yearly_revenue_diff"].sum()*100,2))
                            .assign(revenue_diff_valor_realizado_pct=lambda x: np.round(x["mean_yearly_revenue_diff"]/x["valor_realizado"]*100,2))
                            .sort_values(by="mean_yearly_revenue_diff"))


2. **Análisis Temporal:**

   - Evaluar las tendencias a lo largo del tiempo, por ejemplo, cómo cambian los ingresos realizados de un mes a otro o de un año a otro.
   


As it can be observed from the table below, in 2017 an outstading event creates a bigger gap in the form of overforecasting, that extends up to 2019, after which it returns to its previous path. This overforecast comes from both an increment in forecast with regards to the previous year and a decrease in realised revenue.

In [ ]:
yearly_total = (cleaned_data_df.groupby(['ano_exercicio'])[['diff_valor_previsto_realizado',"valor_previsto_atualizado","valor_realizado"]].sum().reset_index()
                                .assign(pct_valor_realizado=lambda x: x["diff_valor_previsto_realizado"]/x["valor_realizado"]*100))
yearly_total

Averaging out, without lookign at the specific years yet, it seems that most months stay on even forecasting, with a slightly low undeforecasting and most months realising the same amount of revenue. 

However, on months January, March and December there seems to be a big overforecasting, it being tremendous in January and more moderate in December. Those dates can be explained as the moments where a forecast is prepared for the upcoming next or current year. The spike from March is harder to explain and could mean another 2 quarter forecast, but it is unlikely given that there are no other quarter forecasts.

In [ ]:
monthly_total = (cleaned_data_df.groupby(['month'])[['diff_valor_previsto_realizado',"valor_previsto_atualizado","valor_realizado"]].sum().reset_index()
                                .assign(pct_valor_realizado=lambda x: x["diff_valor_previsto_realizado"]/x["valor_realizado"]*100))
monthly_total


3. **Identificación de Discrepancias:**

   - Investigar las categorías con mayor diferencia entre lo previsto y lo realizado, identificando patrones en la subejecución o sobre ejecución.
   


Funny enough, the biggest disruptions created from overforecasting come from the economy ministry, which is however explained its bigger contribution to the national arcs in the form of revenue collection.

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'nome_orgao_superior','nome_orgao'])[['diff_valor_previsto_realizado',"valor_realizado"]].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby(['nome_orgao_superior','nome_orgao'])[['diff_valor_previsto_realizado',"valor_realizado"]].mean().reset_index()
mean_yearly_revenue_category.columns = ["nome_orgao_superior","nome_orgao","mean_yearly_revenue_diff","valor_realizado"]

(mean_yearly_revenue_category.assign(revenue_diff_pct= lambda x: np.round(x["mean_yearly_revenue_diff"]/x["mean_yearly_revenue_diff"].sum()*100,2))
                            .assign(revenue_diff_valor_realizado_pct=lambda x: np.round(x["mean_yearly_revenue_diff"]/x["valor_realizado"]*100,2))
                            .sort_values(by="mean_yearly_revenue_diff")).head(10)

#### Discrepancies between receitas intra_orcamentaria

The impact of intra_orcamentaria is very low.

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'intra_orcamentaria'])[['diff_valor_previsto_realizado',"valor_realizado"]].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby('intra_orcamentaria')[['diff_valor_previsto_realizado',"valor_realizado"]].mean().reset_index()
mean_yearly_revenue_category.columns = ["intra_orcamentaria","mean_yearly_revenue_diff","valor_realizado"]

(mean_yearly_revenue_category.assign(revenue_diff_pct= lambda x: np.round(x["mean_yearly_revenue_diff"]/x["mean_yearly_revenue_diff"].sum()*100,2))
                            .assign(revenue_diff_valor_realizado_pct=lambda x: np.round(x["mean_yearly_revenue_diff"]/x["valor_realizado"]*100,2))
                            .sort_values(by="mean_yearly_revenue_diff"))

### Evaluation of situation 2017

Between 2016 and 2017 there is a decrease in revenue. The impact is of about 0.31 billions of Brazilian reales. Inspecting throuhg the top receitas it could be possible to find the biggest contributors to this decrease:

In [ ]:
top5_especies = contribution_especie[:5].index.to_list()
filter_top5_contributors = cleaned_data_df["especie_receita"].isin(top5_especies)
filter_years = cleaned_data_df["ano_exercicio"].between(2016,2020,inclusive="both")
pivot_table = cleaned_data_df[filter_top5_contributors & filter_years].pivot_table(
    index=['especie_receita'], 
    columns='ano_exercicio', 
    values='valor_realizado', 
    aggfunc='sum'
)
pivot_table.sort_values(by=2016, ascending=False)

In [ ]:
sum(pivot_table.loc[["operacoes_de_credito_mercado_interno"],2016] - pivot_table.loc[["operacoes_de_credito_mercado_interno"],2017])/10**12


The result is that the revenue loss coming from the results of the bank of brasil and the operations of internal market credit had not been forecasted.


## Fase 4: Visualización de Datos

1. **Gráficos de Barras y Líneas:**

   - Crear gráficos que muestren la comparación entre ingresos previstos, lanzados y realizados para cada categoría.


In [481]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
top5_filter = cleaned_data_df["especie_receita"].isin(contribution_especie.index[:5].to_list())
top5_categories_melted = pd.melt(cleaned_data_df[top5_filter], id_vars=["ano_exercicio","mes","especie_receita"] , value_vars=['valor_previsto_atualizado', 'valor_lancado','valor_realizado'])

plt.figure(figsize=(20,10))

plt.suptitle("Forecasted VS realised")
sns.barplot(data=top5_categories_melted,
            x="especie_receita",
            y="value",
        hue= "variable",
        estimator="sum",
        order=top5_especies)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,20))

summarized_df = cleaned_data_df.groupby('especie_receita').agg({
                                'valor_previsto_atualizado': 'sum',
                                'valor_realizado': 'sum'
                            }).reset_index()


ordered_df = summarized_df.sort_values(by='valor_previsto_atualizado')
my_range = range(1, len(ordered_df.index) + 1)

plt.hlines(y=my_range, xmin=ordered_df['valor_previsto_atualizado'], xmax=ordered_df['valor_realizado'], color='grey', alpha=0.4, zorder=1)
plt.scatter(ordered_df['valor_previsto_atualizado'], my_range, color='skyblue', alpha=1, label='Valor Previsto Atualizado')
plt.scatter(ordered_df['valor_realizado'], my_range, color='lightgreen', alpha=1, label='Valor Realizado')


plt.legend()
plt.yticks(my_range, ordered_df['especie_receita'])
plt.title("Comparison of Valor Previsto Atualizado and Valor Realizado", loc='left')
plt.xlabel('Value')
plt.ylabel('Especie Receitas')
plt.show()



   - Graficar la evolución temporal de los ingresos realizados y previstos.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

plt.suptitle("Forecasted VS realised")
sns.lineplot(data=cleaned_data_df,
            x="ano_exercicio",
            y="valor_previsto_atualizado",
        marker = "o", 
        linewidth = 2, 
        label="forecasted",
        color="blue", 
        linestyle = "dashed",
        estimator="sum",
        errorbar=None)
sns.lineplot(data=cleaned_data_df,
    x="ano_exercicio",
    y="valor_realizado",
    marker = "o", 
    linewidth = 2, 
    label="realised", 
    color="orange", 
    linestyle = "dashed",
    estimator="sum",
    errorbar=None)

plt.tight_layout()
plt.show()

In [ ]:

plt.figure(figsize=(20,10))
plt.suptitle("Forecasted vs Realized - Stacked Area Chart")

data_pivot = cleaned_data_df[top5_filter].pivot_table(
    index="ano_exercicio", 
    columns="especie_receita", 
    values="valor_realizado", 
    aggfunc="sum"
).fillna(0)

plt.stackplot(data_pivot.index, data_pivot.T, labels=data_pivot.columns)

plt.xlabel("Year")
plt.ylabel("Realized Value")
plt.legend(loc="upper left")

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axes = plt.subplots(5,2,figsize=(20,20))

axes = axes.flat

plt.suptitle("Forecasted VS realised")
for ax, year in enumerate(cleaned_data_df["ano_exercicio"].unique()):
    axes[ax].set_title(year)
    sns.lineplot(data=cleaned_data_df[cleaned_data_df["ano_exercicio"]==year],
             x="month",
             y="valor_previsto_atualizado",
            marker = "o", 
            linewidth = 1, 
            label="forecasted",
            color="blue", 
            linestyle = "dashed", 
            errorbar = None,
            ax=axes[ax],
            estimator="sum")
    sns.lineplot(data=cleaned_data_df[cleaned_data_df["ano_exercicio"]==year],
        x="month",
        y="valor_realizado",
        marker = "o", 
        linewidth = 1, 
        label="realised", 
        color="orange", 
        linestyle = "dashed", 
        errorbar = None,
        ax=axes[ax],
        estimator="sum")
    axes[ax].tick_params(axis='x', labelrotation=45)

plt.tight_layout()
plt.show()


2. **Diagramas de Caja:**

   - Evaluar la dispersión de las diferencias entre los valores previstos y realizados en diferentes categorías.
  
Los problemas concretos que te han pedido resolver son:

1.	**Desviaciones entre lo previsto y lo recaudado**: Determinar en qué categorías económicas o tipos de ingresos las diferencias son más pronunciadas.

2.	**Evolución temporal de la recaudación**: Identificar cómo han cambiado las previsiones y recaudaciones año a año, y si existen patrones temporales, como meses específicos donde hay mayores discrepancias.

3.	**Rendimiento por órgano y unidad gestora**: Evaluar qué órganos o unidades gestoras son más eficientes en términos de alcanzar las metas de recaudación y cuáles presentan consistentemente una baja ejecución.

Análisis Temporal:

Evaluar las tendencias a lo largo del tiempo, por ejemplo, cómo cambian los ingresos realizados de un mes a otro o de un año a otro.

Annual total forecast vs realised

Forecast VS. realised, monthly data per year

### Comparison between forcasted, launched and realised

1. **Distribución de Ingresos por Categoría Económica:**

   - Analizar las categorías de ingresos más significativas y su participación en los ingresos totales.



   - Calcular la diferencia promedio entre ingresos previstos y realizados por cada categoría.

2. **Análisis Temporal:**

   - Evaluar las tendencias a lo largo del tiempo, por ejemplo, cómo cambian los ingresos realizados de un mes a otro o de un año a otro.

3. **Identificación de Discrepancias:**

   - Investigar las categorías con mayor diferencia entre lo previsto y lo realizado, identificando patrones en la subejecución o sobre ejecución.

## Fase 4: Visualización de Datos

1. **Gráficos de Barras y Líneas:**

   - Crear gráficos que muestren la comparación entre ingresos previstos, lanzados y realizados para cada categoría.

   - Graficar la evolución temporal de los ingresos realizados y previstos.

2. **Diagramas de Caja:**

   - Evaluar la dispersión de las diferencias entre los valores previstos y realizados en diferentes categorías.
  
Los problemas concretos que te han pedido resolver son:

1.	**Desviaciones entre lo previsto y lo recaudado**: Determinar en qué categorías económicas o tipos de ingresos las diferencias son más pronunciadas.

2.	**Evolución temporal de la recaudación**: Identificar cómo han cambiado las previsiones y recaudaciones año a año, y si existen patrones temporales, como meses específicos donde hay mayores discrepancias.

3.	**Rendimiento por órgano y unidad gestora**: Evaluar qué órganos o unidades gestoras son más eficientes en términos de alcanzar las metas de recaudación y cuáles presentan consistentemente una baja ejecución.

### Distributions

In [487]:
value_features = ["valor_previsto_atualizado","valor_lancado","valor_realizado","percentual_realizado","diff_valor_previsto_realizado"]
contribution_especie_top10 = contribution_especie[:10].index.to_list()

In [ ]:
fig, axes = plt.subplots(5,2,figsize=(20,10), sharex=True)

axes = axes.flat

plt.suptitle("Histogram per type of valor - logarithmic scale")
for ax, especie in enumerate(contribution_especie_top10):
    # axes[ax].set_xscale("log")
    axes[ax].set_title(especie.capitalize())
    sns.boxplot(data = cleaned_data_df.query('especie_receita == @especie'), x="diff_valor_previsto_realizado", ax=axes[ax])

plt.tight_layout()
plt.show()

In [ ]:
cleaned_data_df.nunique()

In [ ]:
fig, axes = plt.subplots(5,2,figsize=(20,10), sharex=True)

axes = axes.flat

plt.suptitle("Histogram per type of valor - logarithmic scale")
for ax, especie in enumerate(cleaned_data_df.nome_orao):
    # axes[ax].set_xscale("log")
    axes[ax].set_title(especie.capitalize())
    sns.boxplot(data = cleaned_data_df.query('especie_receita == @especie'), x="diff_valor_previsto_realizado", ax=axes[ax])

plt.tight_layout()
plt.show()

In [ ]:
pd.set_option("display.max_rows",65)
contribution_especie = cleaned_data_df.groupby(["nome_unidade_gestora"])[["valor_realizado"]].sum().assign(valor_realizado_pct=lambda x: np.round(x["valor_realizado"]/cleaned_data_df["valor_realizado"].sum()*100,2)).sort_values("valor_realizado",ascending=False)
contribution_especie[0:10]

In [ ]:
pd.set_option("display.max_rows",65)
contribution_especie = cleaned_data_df.groupby(["nome_orgao"])[["valor_realizado"]].sum().assign(valor_realizado_pct=lambda x: np.round(x["valor_realizado"]/cleaned_data_df["valor_realizado"].sum()*100,2)).sort_values("valor_realizado",ascending=False)
contribution_especie[0:10]

In general, it seems that smaller, non financially related entities perform better than bigger ones.

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'nome_orgao'])[['diff_valor_previsto_realizado',"valor_realizado"]].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby('nome_orgao')[['diff_valor_previsto_realizado',"valor_realizado"]].mean().reset_index()
mean_yearly_revenue_category.columns = ["nome_orgao","mean_yearly_revenue_diff","valor_realizado"]

(mean_yearly_revenue_category.assign(revenue_diff_pct= lambda x: np.round(x["mean_yearly_revenue_diff"]/x["mean_yearly_revenue_diff"].sum()*100,2))
                            .assign(revenue_diff_valor_realizado_pct=lambda x: np.round(x["mean_yearly_revenue_diff"]/x["valor_realizado"]*100,2))
                            .sort_values(by="mean_yearly_revenue_diff"))

In [ ]:
yearly_total_category = cleaned_data_df.groupby(['ano_exercicio', 'nome_unidade_gestora'])[['diff_valor_previsto_realizado',"valor_realizado"]].sum().reset_index()
mean_yearly_revenue_category = yearly_total_category.groupby('nome_unidade_gestora')[['diff_valor_previsto_realizado',"valor_realizado"]].mean().reset_index()
mean_yearly_revenue_category.columns = ["nome_unidade_gestora","mean_yearly_revenue_diff","valor_realizado"]

(mean_yearly_revenue_category.assign(revenue_diff_pct= lambda x: np.round(x["mean_yearly_revenue_diff"]/x["mean_yearly_revenue_diff"].sum()*100,2))
                            .assign(revenue_diff_valor_realizado_pct=lambda x: np.round(x["mean_yearly_revenue_diff"]/x["valor_realizado"]*100,2))
                            .sort_values(by="mean_yearly_revenue_diff"))